If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets as well as other dependencies. Uncomment the following cell and run it.

In [1]:
# ! pip install datasets transformers rouge-score nltk


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

Then you need to install Git-LFS. Uncomment the following instructions:

In [2]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [3]:
import transformers

print(transformers.__version__)

4.28.1


/root/mosaic_ml/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

# Fine-tuning a model on a summarization task

In [3]:
model_checkpoint = "google/flan-t5-xl"

## Loading the dataset

In [5]:
import json

with open("./app/data/data_flant5_en.json", "r") as f:
    data = json.load(f)


In [6]:
from torch.utils.data import Dataset
from transformers import AutoTokenizer

class FlanDataset(Dataset):
    def __init__(self, data, model_name="google/flan-t5-large"):
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]

        prefix = example["prefix"]
        text = example["input"]
        simplified_text = example["output"]

        inputs = self.tokenizer(
            prefix + " "  + text,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                simplified_text,
                max_length=512,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
            )

        inputs["labels"] = labels["input_ids"]

        for k, v in inputs.items():
            inputs[k] = v.squeeze()

        return inputs

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.DataFrame(data)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["prefix"], shuffle=True)
val_df, test_df = train_test_split(val_df, test_size=0.5, random_state=42, stratify=val_df["prefix"], shuffle=True)

train_dataset = FlanDataset(train_df.to_dict("records"))
val_dataset = FlanDataset(val_df.to_dict("records"))
test_dataset = FlanDataset(test_df.to_dict("records"))

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [8]:
from datasets import load_metric
metric = load_metric("rouge")

/tmp/ipykernel_1491113/2721381364.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [9]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

In [10]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [4]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.72s/it]


In [5]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

Note that  we don't get a warning like in our classification example. This means we used all the weights of the pretrained model and there is no randomly initialized head in this case.

To instantiate a `Seq2SeqTrainer`, we will need to define three more things. The most important is the [`Seq2SeqTrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [12]:
batch_size = 2
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-en",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    dataloader_num_workers=8,
    gradient_accumulation_steps=8    
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the cell and customize the weight decay. Since the `Seq2SeqTrainer` will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the `predict_with_generate` option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/t5-finetuned-xsum"` or `"huggingface/t5-finetuned-xsum"`).

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [14]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [15]:
import os
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


We can now finetune our model by just calling the `train` method:

In [33]:
import warnings
warnings.filterwarnings("ignore")
trainer.train()

AttributeError: module 'wandb' has no attribute 'run'

In [17]:
out = trainer.predict(test_dataset)

In [1]:
model

NameError: name 'model' is not defined

In [28]:
out[0].shape

NameError: name 'out' is not defined

#### Inference

In [5]:
def predict(input_text):
    prefix = "simplify:"
    input_text = prefix + " " + input_text
    inputs = tokenizer(input_text, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)
    predicted_abstract_ids = model.generate(input_ids, max_length=512)
    return tokenizer.batch_decode(predicted_abstract_ids)

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# model = AutoModelForSeq2SeqLM.from_pretrained("/home/osama/projects/nlp702_final_project/stanford_alpaca/runs/exp_t5/checkpoint-81")
model.to("cuda:1")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl")

In [3]:
from transformers import pipeline

summarizer = pipeline("text2text-generation", model="/home/osama/projects/nlp702_final_project/stanford_alpaca/runs/exp_t5/checkpoint-81", tokenizer=tokenizer, device=0)


In [4]:
summarizer("simplify: Patient education using virtual reality increases knowledge and positive experience for breast cancer patients undergoing radiation therapy.\nImproved access to technology in the radiation therapy (RT) workforce education has resulted in opportunities for innovative patient education methods. This study investigated the impact of a newly developed education tool using the Virtual Environment for Radiotherapy Training (VERT) system on patients' RT knowledge and anxiety.", max_length=512, min_length=30)

[{'generated_text': ''}]

In [9]:
predict("Patient education using virtual reality increases knowledge and positive experience for breast cancer patients undergoing radiation therapy.\nImproved access to technology in the radiation therapy (RT) workforce education has resulted in opportunities for innovative patient education methods. This study investigated the impact of a newly developed education tool using the Virtual Environment for Radiotherapy Training (VERT) system on patients' RT knowledge and anxiety.")

['<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [26]:
import pickle

with open("out.pkl", "wb") as f:
    pickle.dump(out, f)

In [40]:
test_df['output'].values[5]

'This paper describes a new way to detect microRNAs without using labels or reagents. The method uses a conducting polymer that is structured by carbon nanotubes. The polymer film is electroactive and responds to the microRNA miR-141, which is a biomarker for prostate cancer. The sensor has a very low detection limit of about 8 fM.'

In [41]:
test_df.values[5]

array(['simplify:',
       'Label-free and reagentless electrochemical detection of microRNAs using a conducting polymer nanostructured by carbon nanotubes: application to prostate cancer biomarker miR-141.\nIn this paper, a label-free and reagentless microRNA sensor based on an interpenetrated network of carbon nanotubes and electroactive polymer is described. The nanostructured polymer film presents very well-defined electroactivity in neutral aqueous medium in the cathodic potential domain from the quinone group embedded in the polymer backbone. Addition of microRNA miR-141 target (prostate cancer biomarker) gives a "signal-on" response, i.e. a current increase due to enhancement of the polymer electroactivity. On the contrary, non-complementary miRNAs such as miR-103 and miR-29b-1 do not lead to any significant current change. A very low detection limit of ca. 8 fM is achieved with this sensor.',
       'This paper describes a new way to detect microRNAs without using labels or rea